In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gaussian_renderer import network_gui
from sgs2.gaussian import GaussianModel
from sgs2.scene import Scene
from sgs2.trainers.grid_trainer import GridTrainer
from sgs2.trainers.simple_trainer import SimpleTrainer
from sgs2.evaluation import evaluate_scene
import os
network_gui.init("127.0.0.1", 6009)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/heng/miniconda3/envs/gsn/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth
('127.0.0.1', 6009)


In [3]:
def make_iteration_callback(scene, folder_path="./output", save_interval=100, name="gaussian_model"):
    os.makedirs(folder_path, exist_ok=True)
    def iteration_callback(iteration, n_gauss, mem_use, model=None):
        if model and (iteration % save_interval == 0):
            model_path = os.path.join(folder_path, f"{name}_{iteration}")
            print(f"Saving model at iteration {iteration} to {model_path}")
            model.export_for_sibr(scene, model_path)
    return iteration_callback

# Dataset 1: Train scene
- Method 1: Simple trainer (10K iterations)
- Method 2: Grid trainer, no loss mask (10K iterations)
- Method 3: Grid trainer, with loss mask (10K iterations)

In [4]:
# Load train scene
train_scene = Scene(source_path="./datasets/train/")

Reading camera 301/301
Loading 301 Train Cameras at scale 1.0
Loading Test Cameras


In [ ]:
# Method 1
model = GaussianModel.create_from_scene(train_scene)
simple_trainer = SimpleTrainer(iterations=10000, iteration_callback=make_iteration_callback(train_scene, name="train_scene_simple", save_interval=1000))
simple = simple_trainer.train(train_scene, model)

In [8]:
grid_model = GaussianModel.create_from_scene(train_scene)
grid_trainer = GridTrainer(iterations=10000, network_gui=True)
grid = grid_trainer.train(train_scene, grid_model)


Number of points at initialisation :  182686
Pre-training gaussians...


100%|██████████| 1000/1000 [00:21<00:00, 45.96it/s]


Computing visibility...


100%|██████████| 8/8 [00:04<00:00,  1.93it/s]


Training submodel 1/8...
Filtered cameras from 301 to 249


...

100%|██████████| 1/1 [00:00<00:00, 107.33it/s]


Training submodel 8/8...
Filtered cameras from 301 to 175


100%|██████████| 1/1 [00:00<00:00, 89.00it/s]


Combining gaussians...
Done.


(Scene(source_path='./datasets/train/', images='images', resolution=-1, data_device='cpu', shuffle=True, on_load_progress=None, eval=False, camera_name_whitelist=None, camera_name_blacklist=None),
 GaussianModel(sh_degree=3, percent_dense=0.01, position_lr_init=0.00016, feature_lr=0.0025, opacity_lr=0.05, scaling_lr=0.005, rotation_lr=0.001, position_lr_final=1.6e-06, spatial_lr_scale=0, position_lr_delay_mult=0.01, position_lr_max_steps=30000, pipeline_params=PipelineParams(debug=False, compute_cov3D_python=False, convert_SHs_python=False)))

In [9]:
grid.export_for_sibr(train_scene, "./output/grid_model3")

Creating output folder at /home/heng/Documents/GitHub/gaussian-splatting/output/grid_model3
